# CVBS Video Capture

This notebook captures 2 seconds of CVBS (composite video) signal using Deep Memory Acquisition (DMA).

## Configuration
- **Target sample rate:** 4fsc = 14.3181818 MS/s (NTSC standard)
- **Actual sample rate:** 15.625 MS/s (Dec 8, closest available)
- **Capture duration:** 2 seconds
- **Expected data size:** ~62.5 MB

## Hardware Setup
- Set input jumpers to **LV** (±1V range) for CVBS signals
- Connect CVBS source to **IN1** (RF input 1)
- Consider 75Ω termination for proper impedance matching

## Output
- Saves raw binary data to `/tmp/cvbs_capture.bin`
- Saves metadata to `/tmp/cvbs_capture_meta.txt`
- Use `visualize_capture.py` on your local machine to view the data

## 1. Initialize Libraries and FPGA

In [ ]:
import time
import os
import array
from rp_overlay import overlay
import rp

# Initialize FPGA overlay and RP library
fpga = overlay()
rp.rp_Init()
print("Red Pitaya initialized")

## 2. Capture Parameters

In [ ]:
# Capture configuration
CAPTURE_DURATION_SEC = 2.0
DECIMATION = rp.RP_DEC_8          # 125 MS/s / 8 = 15.625 MS/s
SAMPLE_RATE = 125e6 / 8           # 15.625 MS/s
TARGET_4FSC = 14.3181818e6        # 4x NTSC color subcarrier

# Calculate buffer size
DATA_SIZE = int(CAPTURE_DURATION_SEC * SAMPLE_RATE)

print(f"Capture duration: {CAPTURE_DURATION_SEC} seconds")
print(f"Sample rate: {SAMPLE_RATE/1e6:.3f} MS/s (target 4fsc: {TARGET_4FSC/1e6:.3f} MS/s)")
print(f"Sample rate error: {((SAMPLE_RATE - TARGET_4FSC) / TARGET_4FSC * 100):+.2f}%")
print(f"Total samples: {DATA_SIZE:,}")
print(f"Data size: {DATA_SIZE * 2 / 1024 / 1024:.1f} MB")

## 3. Check DMA Memory Region

In [ ]:
# Get available DMA memory region
memoryRegion = rp.rp_AcqAxiGetMemoryRegion()
g_adc_axi_start = memoryRegion[1]
g_adc_axi_size = memoryRegion[2]

print(f"DMA Memory Region:")
print(f"  Start address: 0x{g_adc_axi_start:08X}")
print(f"  Size (bytes): {g_adc_axi_size:,} ({g_adc_axi_size / 1024 / 1024:.1f} MB)")
print(f"  Max samples (16-bit): {g_adc_axi_size // 2:,}")

# Verify we have enough memory
required_bytes = DATA_SIZE * 2  # 16-bit samples
if required_bytes > g_adc_axi_size:
    print(f"\nWARNING: Required {required_bytes:,} bytes but only {g_adc_axi_size:,} available!")
    print("Reducing capture duration...")
    DATA_SIZE = g_adc_axi_size // 2
    CAPTURE_DURATION_SEC = DATA_SIZE / SAMPLE_RATE
    print(f"New capture duration: {CAPTURE_DURATION_SEC:.2f} seconds")
else:
    print(f"\nMemory OK: Need {required_bytes/1024/1024:.1f} MB, have {g_adc_axi_size/1024/1024:.1f} MB")

## 4. Configure and Start DMA Acquisition

In [ ]:
# Reset acquisition
rp.rp_AcqReset()

# Configure DMA
rp.rp_AcqAxiSetDecimationFactor(DECIMATION)
print(f"Decimation set to: {DECIMATION}")

# Set trigger delay (capture DATA_SIZE samples after trigger)
rp.rp_AcqAxiSetTriggerDelay(rp.RP_CH_1, DATA_SIZE)

# Configure buffer - use channel 1 only, full memory region
rp.rp_AcqAxiSetBufferSamples(rp.RP_CH_1, g_adc_axi_start, DATA_SIZE)
print(f"Buffer configured: {DATA_SIZE:,} samples at 0x{g_adc_axi_start:08X}")

# Enable DMA on channel 1
rp.rp_AcqAxiEnable(rp.RP_CH_1, True)
print("DMA enabled on CH1")

In [ ]:
# Start acquisition
print("Starting acquisition...")
rp.rp_AcqStart()

# Brief delay to let acquisition stabilize
time.sleep(0.1)

# Trigger immediately (RP_TRIG_SRC_NOW)
print("Triggering NOW...")
rp.rp_AcqSetTriggerSrc(rp.RP_TRIG_SRC_NOW)

# Wait for trigger to occur
while True:
    trig_state = rp.rp_AcqGetTriggerState()[1]
    if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
        print("Triggered!")
        break

# Wait for buffer to fill
print(f"Capturing {CAPTURE_DURATION_SEC} seconds of data...")
start_time = time.time()

while True:
    fillState = rp.rp_AcqAxiGetBufferFillState(rp.RP_CH_1)[1]
    if fillState:
        break
    elapsed = time.time() - start_time
    if elapsed > CAPTURE_DURATION_SEC + 5:  # Timeout after expected duration + 5s
        print(f"WARNING: Timeout after {elapsed:.1f}s")
        break

capture_time = time.time() - start_time
print(f"Buffer filled in {capture_time:.2f} seconds")

# Stop acquisition
rp.rp_AcqStop()
print("Acquisition stopped")

## 5. Read Data and Save to File

In [ ]:
# Get write pointer at trigger
posChA = rp.rp_AcqAxiGetWritePointerAtTrig(rp.RP_CH_1)[1]
print(f"Write pointer at trigger: {posChA}")

# Output files
output_filename = "/tmp/cvbs_capture.bin"
metadata_filename = "/tmp/cvbs_capture_meta.txt"

# Remove old files
for f in [output_filename, metadata_filename]:
    if os.path.exists(f):
        os.remove(f)

# Try to use ctypes for fast buffer access
import ctypes

CHUNK_SIZE = 4 * 1024 * 1024  # 4M samples per chunk
buff = rp.i16Buffer(CHUNK_SIZE)

# Check if we can get a pointer from the buffer for direct memory access
buff_ptr = buff.cast()
print(f"Buffer pointer type: {type(buff_ptr)}")

print(f"Reading and saving data to {output_filename}...")
read_start = time.time()
samples_written = 0
read_pos = posChA

with open(output_filename, 'wb') as f:
    while samples_written < DATA_SIZE:
        chunk_size = min(CHUNK_SIZE, DATA_SIZE - samples_written)
        
        # Read chunk from DMA
        rp.rp_AcqAxiGetDataRaw(rp.RP_CH_1, read_pos, chunk_size, buff_ptr)
        
        # Try to get raw bytes using ctypes (avoids Python loop)
        try:
            # Get pointer address and read bytes directly
            ptr_addr = int(buff_ptr)
            byte_data = ctypes.string_at(ptr_addr, chunk_size * 2)  # 2 bytes per int16
            f.write(byte_data)
        except (TypeError, ValueError) as e:
            # Fallback to array method if ctypes fails
            print(f"ctypes failed ({e}), using slow method...")
            arr = array.array('h', (buff[i] for i in range(chunk_size)))
            f.write(arr.tobytes())
        
        read_pos += chunk_size
        samples_written += chunk_size
        pct = samples_written / DATA_SIZE * 100
        print(f"  {pct:.0f}% ({samples_written:,} samples)")

read_time = time.time() - read_start
print(f"\nData saved: {DATA_SIZE:,} samples in {read_time:.1f}s")
print(f"Write speed: {DATA_SIZE * 2 / read_time / 1024 / 1024:.1f} MB/s")
print(f"File size: {os.path.getsize(output_filename) / 1024 / 1024:.1f} MB")

In [ ]:
# Save metadata as plain text
with open(metadata_filename, 'w') as f:
    f.write(f"sample_rate={SAMPLE_RATE}\n")
    f.write(f"target_sample_rate_4fsc={TARGET_4FSC}\n")
    f.write(f"decimation=8\n")
    f.write(f"capture_duration_sec={CAPTURE_DURATION_SEC}\n")
    f.write(f"num_samples={DATA_SIZE}\n")
    f.write(f"timestamp={time.time()}\n")
    f.write(f"dtype=int16\n")
    f.write(f"bits=14\n")
    f.write(f"endian=little\n")

print(f"Metadata saved: {metadata_filename}")
print(f"\nTo retrieve via SCP:")
print(f"  scp root@192.168.0.6:/tmp/cvbs_capture.bin .")
print(f"  scp root@192.168.0.6:/tmp/cvbs_capture_meta.txt .")
print(f"\nThen run locally:")
print(f"  python visualize_capture.py cvbs_capture.bin")

## 6. Cleanup

In [ ]:
# Disable DMA and release resources
rp.rp_AcqAxiEnable(rp.RP_CH_1, False)
rp.rp_Release()
print("Resources released")